# Mover información cruda a aumentada

## Importamos librerías

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
import uuid

## Creamos variables claves del proyecto

In [0]:
# Variables de ubicación de archivos
dl_location = 'abfss://dataengineering@datarepositoryg4jdurand1.dfs.core.windows.net/'
raw_location = dl_location + 'RAW/'

uc_location_aumented = 'medallion1_architecture.aumented.'

# Otras variables
date_format = 'dd/MM/yyyy'

## Creamos schemas para las diferentes tablas a leer

### Schema para las tablas de circuitos

In [0]:


# 
df_circuits_schema = StructType(
  fields=[
    StructField("circuitId", IntegerType(), True),
    StructField("circuitRef", StringType(), True),
    StructField("name", StringType(), True),
    StructField("location", StringType(), True),
    StructField("country", StringType(), True),
    StructField("lat", DoubleType(), True),
    StructField("lng", DoubleType(), True),
    StructField("alt", IntegerType(), True),
    StructField("url", StringType(), True)
])


### Schema para las tablas de razas

In [0]:
df_races_schema = StructType(
  fields=[
    StructField("raceId", IntegerType(), True),
    StructField("year", IntegerType(), True),
    StructField("round", IntegerType(), True),
    StructField("circuitId", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("date", DateType(), True),  # El tipo de dato es DATE
    StructField("time", StringType(), True),  # Mantenemos STRING por si incluye hora
    StructField("url", StringType(), True)
])

### Schema para las tablas de constructor

In [0]:
df_constructor_schema = StructType(
  fields=[
    StructField("constructorId", IntegerType(), True),
    StructField("constructorRef", StringType(), True),
    StructField("name", StringType(), True),
    StructField("nationality", StringType(), True),
    StructField("url", StringType(), True)
])

### Schema para las tablas de resultados

In [0]:
df_result_schema = StructType(
  fields=[
    StructField("resultId", IntegerType(), True),
    StructField("raceId", IntegerType(), True),
    StructField("driverId", IntegerType(), True),
    StructField("constructorId", IntegerType(), True),
    StructField("number", IntegerType(), True),
    StructField("grid", IntegerType(), True),
    StructField("position", IntegerType(), True),
    StructField("positionText", StringType(), True),
    StructField("positionOrder", IntegerType(), True),
    StructField("points", IntegerType(), True),
    StructField("laps", IntegerType(), True),
    StructField("time", StringType(), True),
    StructField("milliseconds", IntegerType(), True),
    StructField("fastestLap", IntegerType(), True),
    StructField("rank", IntegerType(), True),
    StructField("fastestLapTime", StringType(), True),
    StructField("fastestLapSpeed", FloatType(), True),
    StructField("statusId", StringType(), True)
])

### Schema para las tablas de conductores


In [0]:

df_driver_schema = StructType(
  fields=[
    StructField("driverId", IntegerType(), True),
    StructField("duration", StringType(), True),
    StructField("lap", IntegerType(), True),
    StructField("milliseconds", IntegerType(), True),
    StructField("raceId", IntegerType(), True),
    StructField("stop", IntegerType(), True),
    StructField("time", StringType(), True)
])

### Schema para las tablas de conductores

In [0]:
df_pit_stops_schema = StructType(
    fields=[
        StructField("driverId", IntegerType(), True),
        StructField("duration", StringType(), True),
        StructField("lap", IntegerType(), True),
        StructField("milliseconds", IntegerType(), True),
        StructField("raceId", IntegerType(), True),
        StructField("stop", IntegerType(), True),
        StructField("time", StringType(), True)
    ]
)

- ### Schema para las tablas de calificación

In [0]:
df_qualifying_schema = StructType(
    fields=[
        StructField("constructorId", IntegerType(), True),
        StructField("driverId", IntegerType(), True),
        StructField("number", IntegerType(), True),
        StructField("position", IntegerType(), True),
        StructField("q1", StringType(), True),
        StructField("q2", StringType(), True),
        StructField("q3", StringType(), True),
        StructField("qualifyId", IntegerType(), True),
        StructField("raceId", IntegerType(), True)
    ]
)

## Unity Catalog

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS medallion_architecture.tracking
COMMENT 'En este schema va a ir guardada la informacion agregada'
;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS medallion_architecture.tracking.summary(
    TRANSACTION_DATE TIMESTAMP,
    FOLDER_NAME STRING,
    UUID STRING
)

## Definición de funciones personalizadas

In [0]:
# Función para convertir las columnas de fecha a formato Date
def converting_to_date(df: DataFrame, colname: str, date_format: str) -> DataFrame:
    df = df.withColumn(
        colname,
        to_date(
            concat(
                substring(col(colname), 7, 2), 
                lit('/'),
                substring(col(colname), 5, 2),
                lit('/'),
                substring(col(colname), 1, 4)
            ),
            date_format
        ) 
    )
    return(df)

In [0]:
# Función para quitar los valores nulos
def cleaning_nulls(df: DataFrame) -> DataFrame:
    df = df.fillna({
        "CUST_CODE": "CUSTXXXXXXXXXX",
        "seg_1": "NA",
        "seg_2": "NA"
    })
    return(df)

In [0]:
def writing_info(df: DataFrame, table_name: str) -> None:
    df.write \
    .mode("append") \
    .format("delta") \
    .saveAsTable(f"{uc_location_aumented}{table_name}")

In [0]:
def  writing_tracking_table(raw_folders: list) -> None:
     
    raw_folders = [(folder,) for folder in raw_folders] 

    uuid_udf = udf(lambda: str(uuid.uuid4()), StringType())
    
    df_tracking = spark.createDataFrame(raw_folders,schema=['FOLDER_NAME'])
    df_tracking = df_tracking.withColumn("UUID", uuid_udf())
    df_tracking = df_tracking.withColumn("TRANSACTION_DATE", current_timestamp())

    df_tracking.write \
        .mode("append") \
        .format("delta") \
        .option("mergeSchema", "true") \
        .saveAsTable("medallion_architecture.tracking.summary")

## Ejecución del código

In [0]:
folders = dbutils.fs.ls(raw_location)
print(folders)

In [0]:
raw_folders = dbutils.fs.ls(raw_location)
raw_folders = [folder[0] for folder in raw_folders]
raw_folders

In [0]:
df_folders = spark.sql("SELECT FOLDER_NAME FROM medallion_architecture.tracking.summary")
processed_folders = df_folders.select("FOLDER_NAME").rdd.flatMap(lambda x: x).collect()
processed_folders

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS medallion_architecture.aumented
COMMENT 'En este schema va a ir guardada la informacion agregada'
;

In [0]:
if not processed_folders:

    df_transactions = (
    spark
    .read
    .format('csv')
    .schema(df_transaction_schema)
    .option('header', True)
    .load(f"{raw_location}/*/transactions_*.csv")
    )

    df_time = (
    spark
    .read
    .format('csv')
    .schema(df_time_schema)
    .option('header', True)
    .load(f"{raw_location}/*/time.csv")
    )

    df_transactions = converting_to_date(df = df_transactions,colname = "SHOP_DATE",date_format = date_format)
    df_transactions = cleaning_nulls(df_transactions)

    writing_info(df=df_transactions, table_name="Transactions")
    writing_info(df=df_time, table_name="Time_table")
    writing_tracking_table(raw_folders)

else:
    for raw_folder in raw_folders:
        if raw_folder in processed_folders:
            print(f"Folder: {raw_folder} already processed")
            pass
        else:
            df_transactions = (
                spark
                .read
                .format('csv')
                .schema(df_transaction_schema)
                .option('header', True)
                .load(f"{raw_folder}transactions_*.csv")
                )

            df_time = (
                spark
                .read
                .format('csv')
                .schema(df_time_schema)
                .option('header', True)
                .load(f"{raw_folder}time.csv")
                )
            
            df_transactions = converting_to_date(df = df_transactions,colname = "SHOP_DATE",date_format = date_format)
            df_transactions = cleaning_nulls(df_transactions)

            writing_info(df=df_transactions, table_name="Transactions")
            writing_info(df=df_time, table_name="Time_table")
            writing_tracking_table([raw_folder])

display(df_transactions)

In [0]:
%sql
--DROP TABLE IF EXISTS medallion_architecture.aumented.transactions;
--DROP TABLE IF EXISTS medallion_architecture.aumented.time_table;
--DROP SCHEMA IF EXISTS medallion_architecture.raw;
--DROP TABLE IF EXISTS medallion_architecture.tracking.summary